<a href="https://colab.research.google.com/github/markeldon32/hello-world/blob/master/TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###To learn evaluating Text with sklearn


In [93]:
import pandas as pd
from scipy.sparse import coo_matrix

# read json into a data frame
df_idf = pd.read_json('https://raw.githubusercontent.com/kavgan/nlp-in-practice/master/tf-idf/data/stackoverflow-data-idf.json', lines=True)

#print Schema
print('Schema:\n\n', df_idf.dtypes)
print('Number of questions, columns=', df_idf.shape)


Schema:

 id                            int64
title                        object
body                         object
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_editor_display_name     object
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
accepted_answer_id          float64
favorite_count              float64
last_edit_date               object
last_editor_user_id         float64
community_owned_date         object
dtype: object
Number of questions, columns= (20000, 19)


In [94]:
import re
def pre_process(text):

  #lowercase
  text=text.lower()
  #remove tags
  text=re.sub('&lt;/?.*?&gt;','&lt;&gt;',text)
  #remove special characters and digits
  text=re.sub('(\\d|\\W)+',' ',text)

  return text

df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text'][2]


'gradle command line p i m trying to run a shell script with gradle i currently have something like this p pre code def test project tasks create test exec commandline bash c bash c my file dir script sh code pre p the problem is that i cannot run this script because i have spaces in my dir name i have tried everything e g p pre code commandline bash c bash c my file dir script sh tokenize commandline bash c bash c my file dir script sh commandline bash c new stringbuilder append bash append c my file dir script sh commandline bash c bash c my file dir script sh file dir file c my file dir script sh commandline bash c bash dir getabsolutepath code pre p im using windows bit and if i use a path without spaces the script runs perfectly therefore the only issue as i can see is how gradle handles spaces p '

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
  """load stop words """

  with open(stop_file_path, 'r', encoding='utf-8') as f:
    stopwords= f.readlines()
    stop_set=set(m.strip() for m in stopwords)
    return frozenset(stop_set)

#load a set of stopwords
stopwords=get_stop_words('/content/drive/My Drive/Colab Notebooks/stopwords.txt')

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of word
#ignore words that appear in 85% of the documents
#eliminate stopwords
cv= CountVectorizer(max_df=0.85, stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['let'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [96]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['let'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [97]:
list(cv.vocabulary_.keys())[:20]

['serializing',
 'private',
 'struct',
 'done',
 'public',
 'class',
 'contains',
 'properties',
 'mostly',
 'string',
 'want',
 'serialize',
 'attempt',
 'stream',
 'disk',
 'using',
 'xmlserializer',
 'get',
 'error',
 'saying']

In [98]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [0]:
#read test docs into dataframe and concatenate title and body
df_test=pd.read_json('https://raw.githubusercontent.com/kavgan/nlp-in-practice/master/tf-idf/data/stackoverflow-test.json',lines=True)
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] = df_test['text'].apply(lambda x:pre_process(x))

#test docs into a list
docs_test=df_test['text'].tolist()


In [100]:
from scipy.sparse import coo_matrix
import numpy as np

# you only need to do this once, as this is a mapping index to
feature_names = cv.get_feature_names()

# get the document which we want to extract the keywords
doc= docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

# sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n = 10
keywords=extract-topn_from_vector(feature_names, sorted_items,10)

#print the results
print('\n=====Doc=====')
print(doc)
print('\n===Keywords===')
for k in keywords:
  print(k,keywords[k])

NameError: ignored